In [1]:
import os 
import sys
from tqdm import tqdm
import pickle
import importlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

module_path = '/home/lun/project-basileus/seq-gan/sgtlstm'
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
if 'sgtlstm' in sys.modules:
    importlib.reload(sys.modules['sgtlstm'])

from sgtlstm.utils import load_fixed_length_sequence_from_pickle, create_dataset
from sgtlstm.utils import create_dataset
from sgtlstm.SeqGan import build_G, build_D
from sgtlstm.pretrain import pretrain_generator, pretrain_discriminator, create_self_regression_data_batch
from sgtlstm.TimeLSTM import TimeLSTM0, TimeLSTM1, TimeLSTM2, TimeLSTM3

In [3]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import LSTM

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Input, LSTM, Embedding, Reshape, Dense
import tensorflow_probability as tfp
from tensorflow_probability import distributions as tfd

In [4]:
BATCH_SIZE = 128
T = 10 + 1
VOCAB = ['END/PADDING', 'INIT', 'start', 'click', 'install']
EVENT_VOCAB_DIM = len(VOCAB)
EMB_DIM = 5
HIDDEN_DIM = 64
K_MIST = 2

END_TOKEN = 0
MAX_TIME = 1024

## Load data 

In [6]:
valid_data_path = '/home/lun/project-basileus/seq-gan/data/fixed_length_with_init_token/valid_sequences.pickle'
invalid_data_path = '/home/lun/project-basileus/seq-gan/data/fixed_length_with_init_token/invalid_sequences.pickle'
gen_seq_path_pretrain = '/home/lun/project-basileus/seq-gan/experiment_results/init_results_after_pretrain/generated_seqs/generated_seqs.pickle'
gen_seq_path_1 = '/home/lun/project-basileus/seq-gan/experiment_results/after_1_epochs/generated_seqs/generated_seqs.pickle'
gen_seq_path_4 = '/home/lun/project-basileus/seq-gan/experiment_results/after_4_epochs/generated_seqs/generated_seqs.pickle'
gen_seq_path_8 = '/home/lun/project-basileus/seq-gan/experiment_results/after_8_epochs/generated_seqs/generated_seqs.pickle'

valid_event_type_seqs, valid_timestamp_seqs = load_fixed_length_sequence_from_pickle(valid_data_path, to_timedelta=True, end_token=0)
invalid_event_type_seqs, invalid_timestamp_seqs = load_fixed_length_sequence_from_pickle(invalid_data_path, to_timedelta=True, end_token=0)

generated_event_type_seqs_pretrain, generated_timestamp_seqs_pretrain = load_fixed_length_sequence_from_pickle(gen_seq_path_pretrain, to_timedelta=True, end_token=None)
generated_event_type_seqs_1, generated_timestamp_seqs_1 = load_fixed_length_sequence_from_pickle(gen_seq_path_1, to_timedelta=True, end_token=None)
generated_event_type_seqs_4, generated_timestamp_seqs_4 = load_fixed_length_sequence_from_pickle(gen_seq_path_4, to_timedelta=True, end_token=None)
generated_event_type_seqs_8, generated_timestamp_seqs_8 = load_fixed_length_sequence_from_pickle(gen_seq_path_8, to_timedelta=True, end_token=None)

## Load D models

In [8]:
def load_saved_D_model(D_save_path):
    discriminator = build_D(
        T = T,
        event_vocab_dim = EVENT_VOCAB_DIM,
        emb_dim = EMB_DIM,
        hidden_dim= HIDDEN_DIM,
        k_mixt = K_MIST
    )
    discriminator.build(input_shape=((BATCH_SIZE, T, 1), (BATCH_SIZE, T, 1)))    
    discriminator.load_weights(D_save_path)
    return discriminator

In [ ]:
pretrained_D_path = '/home/lun/project-basileus/seq-gan/gan_model_weights/pretrain_40000_disc/model.tf'
D_path_1 = '/home/lun/project-basileus/seq-gan/experiment_results/after_1_epochs/disc_weights/disc_model.tf'
D_path_4 = '/home/lun/project-basileus/seq-gan/experiment_results/after_4_epochs/disc_weights/disc_model.tf'
D_path_8 = '/home/lun/project-basileus/seq-gan/experiment_results/after_8_epochs/disc_weights/disc_model.tf'

In [9]:
D_pretrain = load_saved_D_model(pretrained_D_path)
D_1 = load_saved_D_model(D_path_1)
D_4 = load_saved_D_model(D_path_4)
D_8 = load_saved_D_model(D_path_8)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 3, expecting 4
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full 

## Predict using pretrained genearator

In [ ]:
true_data = (valid_event_type_seqs[:1000,:,:], valid_timestamp_seqs[:1000,:,:])

In [15]:
def D_predict_analyze(D, event_type_seqs, timestamp_seqs):
    pred_prob, pred_gaussian_log, _ = D((event_type_seqs, timestamp_seqs))    
    return pred_prob, pred_gaussian_log

### pre-train

In [21]:
pred_prob, pred_gaussian_log = D_predict_analyze(D_pretrain, generated_event_type_seqs_pretrain, generated_timestamp_seqs_pretrain)
print('Generated :', sum(pred_prob.numpy() > 0.5))

pred_prob, pred_gaussian_log = D_predict_analyze(D_pretrain, valid_event_type_seqs[:1000,:,:], valid_timestamp_seqs[:1000,:,:])
print('True :', sum(pred_prob.numpy() > 0.5))

[766]
[661]


### 1 epoch

In [23]:
pred_prob, pred_gaussian_log = D_predict_analyze(D_1, generated_event_type_seqs_1, generated_timestamp_seqs_1)
print('Generated :', sum(pred_prob.numpy() > 0.5))

pred_prob, pred_gaussian_log = D_predict_analyze(D_1, valid_event_type_seqs[:1000,:,:], valid_timestamp_seqs[:1000,:,:])
print('True :', sum(pred_prob.numpy() > 0.5))

Generated : [1]
True : [992]


### 4 epoch

In [25]:
pred_prob, pred_gaussian_log = D_predict_analyze(D_4, generated_event_type_seqs_4, generated_timestamp_seqs_4)
print('Generated :', sum(pred_prob.numpy() > 0.5))

pred_prob, pred_gaussian_log = D_predict_analyze(D_4, valid_event_type_seqs[:1000,:,:], valid_timestamp_seqs[:1000,:,:])
print('True :', sum(pred_prob.numpy() > 0.5))

Generated : [1]
True : [1000]


### 8 epoch

In [27]:
pred_prob, pred_gaussian_log = D_predict_analyze(D_8, generated_event_type_seqs_8, generated_timestamp_seqs_8)
print('Generated :', sum(pred_prob.numpy() > 0.5))

pred_prob, pred_gaussian_log = D_predict_analyze(D_8, valid_event_type_seqs[:1000,:,:], valid_timestamp_seqs[:1000,:,:])
print('True :', sum(pred_prob.numpy() > 0.5))

Generated : [0]
True : [1000]
